In [ ]:
import cv2
import os
import pandas as pd

dirr1 = r"/content/Malignant_Images"
csv = r"/content/Malignant.csv"
c = 0

j = 0
df1 = pd.DataFrame(columns=['class', 'Solidity', 'sift_keypoints', 'wh_ratio', 'ecl_ratio', 'harris_corner', 'circularity'])

for i in os.listdir(dirr1):
    path = os.path.join(dirr1, i)
    img = cv2.imread(path)
    print('working on img', i)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    ret, thresh = cv2.threshold(gray, 150, 255, 0)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    if contours:  # Check if contours are found
        cnt = max(contours, key=cv2.contourArea)
        area = cv2.contourArea(cnt)
        hull = cv2.convexHull(cnt)
        hull_area = cv2.contourArea(hull)

        if hull_area == 0:
            print("Warning: Convex hull area is zero for image", i)
            continue
        Solidity = float(area) / hull_area
        perimeter = cv2.arcLength(cnt, True)
        circularity = ((4 * 3.1416 * area) / (perimeter ** 2))

        sift = cv2.SIFT_create()
        kp = sift.detect(gray, None)
        sift_keypoints = len(kp)

        x, y, w, h = cv2.boundingRect(cnt)
        wh_ratio = w / h

        M = cv2.moments(thresh)
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        center_coordinates = (cX, cY)
        w = int(w / 2)
        h = int(h / 2)
        axesLength = (w, h)
        angle = 0
        startAngle = 0
        endAngle = 360
        color = (0, 0, 255)
        thickness = -1
        eclipse = cv2.ellipse(img, center_coordinates, axesLength, angle, startAngle, endAngle, color, thickness)
        lower_blue = np.array([0, 0, 222])
        upper_blue = np.array([0, 0, 255])
        mask_blue = cv2.inRange(eclipse, lower_blue, upper_blue)
        contours, hierarchy = cv2.findContours(mask_blue, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        try:
            ecl_cnt = max(contours, key=cv2.contourArea)
            ecl_area = cv2.contourArea(ecl_cnt)
            ecl_ratio = area / ecl_area,l
        except:
            ecl_ratio = 0

        gray = np.float32(gray)
        dst = cv2.cornerHarris(gray, 5, 3, 0.04)
        ret, dst = cv2.threshold(dst, 0.1 * dst.max(), 255, 0)
        dst = np.uint8(dst)
        ret, labels, stats, centroids = cv2.connectedComponentsWithStats(dst)
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
        corners = cv2.cornerSubPix(gray, np.float32(centroids), (5, 5), (-1, -1), criteria)
        harris_corner = len(corners)

        df1.loc[j] = ({'class': c, 'Solidity': Solidity, 'sift_keypoints': sift_keypoints, 'wh_ratio': wh_ratio,
                       'ecl_ratio': ecl_ratio, 'harris_corner': harris_corner, 'circularity': circularity})
        j += 1
    else:
        print("No contours found for image", i)

frames = df1
frames.to_csv(csv, index=False)
